## Working with Images

In [6]:
import torch

In [16]:
import warnings
warnings.simplefilter("ignore")

In [3]:
import imageio

In [4]:
img_arr = imageio.imread("git_rep/data/p1ch4/image-dog/bobby.jpg")

C:\Users\jnrah\AppData\Local\Temp\ipykernel_6388\3949246334.py:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_arr = imageio.imread("git_rep/data/p1ch4/image-dog/bobby.jpg")


In [5]:
img_arr.shape

(720, 1280, 3)

In [7]:
img = torch.from_numpy(img_arr)
out = img.permute(2,0,1)

In [9]:
batch_size = 3

In [17]:
batch = torch.zeros(batch_size, 3, 256,256, dtype = torch.uint8)

In [11]:
import os

In [12]:
data_dir = "git_rep/data/p1ch4/image-cats/"

In [13]:
filenames = [name for name in os.listdir(data_dir) if(os.path.splitext(name)[-1] == '.png')]

In [18]:
for i, filename in enumerate(filenames):
    img_arr = imageio.imread(os.path.join(data_dir, filename))
    img_t = torch.from_numpy(img_arr)
    img_t = img_t.permute(2,0,1)
    img_t = img_t[:3]
    batch[i] = img_t

In [25]:
batch = batch.float()
n_channels = batch.shape[1]
for c in range(n_channels):
    mean = torch.mean(batch[:,c])
    std = torch.std(batch[:,c])
    batch[:,c] = (batch[:,c] - mean)/std

## Volumetric Data

In [27]:
dir_path = "git_rep/data/p1ch4/volumetric-dicom/2-LUNG 3.0  B70f-04083/"

In [28]:
vol_arr = imageio.volread(dir_path, 'DICOM')

Reading DICOM (examining files): 1/99 files (1.0%7/99 files (7.1%12/99 files (12.1%17/99 files (17.2%21/99 files (21.2%24/99 files (24.2%29/99 files (29.3%35/99 files (35.4%41/99 files (41.4%48/99 files (48.5%54/99 files (54.5%60/99 files (60.6%65/99 files (65.7%69/99 files (69.7%73/99 files (73.7%77/99 files (77.8%84/99 files (84.8%90/99 files (90.9%94/99 files (94.9%99/99 files (100.099/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (loading data): 2/99  (2.015/99  (15.226/99  (26.335/99  (35.446/99  (46.557/99  (57.674/99  (74.790/99  (90.999/99  (100.0%)


In [30]:
vol_arr.shape

(99, 512, 512)

In [31]:
vol = torch.from_numpy(vol_arr).float()

In [32]:
vol = torch.unsqueeze(vol,0)

In [33]:
vol.shape

torch.Size([1, 99, 512, 512])

## Representing Tabular Data

In [36]:
import csv
import numpy as np

In [37]:
wine_path = "git_rep/data/p1ch4/tabular-wine/winequality-white.csv"

In [38]:
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";", skiprows=1)

In [39]:
wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [40]:
col_list = next(csv.reader(open(wine_path),delimiter=";"))

In [41]:
wineq_numpy.shape, col_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [42]:
wineq = torch.from_numpy(wineq_numpy)

In [43]:
wineq.shape, wineq.dtype

(torch.Size([4898, 12]), torch.float32)

In [44]:
data = wineq[:,:-1]

In [45]:
data, data.shape

(tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
         [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
         [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
         ...,
         [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
         [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
         [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]]),
 torch.Size([4898, 11]))

In [46]:
target = wineq[:,-1]

In [48]:
target, target.shape

(tensor([6., 6., 6.,  ..., 6., 7., 6.]), torch.Size([4898]))

In [49]:
target = wineq[:,-1].long()
target

tensor([6, 6, 6,  ..., 6, 7, 6])

In [50]:
target_onehot = torch.zeros(target.shape[0],10)

In [51]:
target_onehot.scatter_(1, target.unsqueeze(1), 1.0)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [53]:
data_mean = torch.mean(data, dim = 0)

In [54]:
data_mean

tensor([6.8548e+00, 2.7824e-01, 3.3419e-01, 6.3914e+00, 4.5772e-02, 3.5308e+01,
        1.3836e+02, 9.9403e-01, 3.1883e+00, 4.8985e-01, 1.0514e+01])

In [55]:
data_var = torch.var(data, dim = 0)

In [56]:
data_var

tensor([7.1211e-01, 1.0160e-02, 1.4646e-02, 2.5726e+01, 4.7733e-04, 2.8924e+02,
        1.8061e+03, 8.9455e-06, 2.2801e-02, 1.3025e-02, 1.5144e+00])

In [57]:
data_normalized = (data - data_mean) / torch.sqrt(data_var)

In [58]:
data_normalized

tensor([[ 1.7208e-01, -8.1761e-02,  2.1326e-01,  ..., -1.2468e+00,
         -3.4915e-01, -1.3930e+00],
        [-6.5743e-01,  2.1587e-01,  4.7996e-02,  ...,  7.3995e-01,
          1.3422e-03, -8.2419e-01],
        [ 1.4756e+00,  1.7450e-02,  5.4378e-01,  ...,  4.7505e-01,
         -4.3677e-01, -3.3663e-01],
        ...,
        [-4.2043e-01, -3.7940e-01, -1.1915e+00,  ..., -1.3130e+00,
         -2.6153e-01, -9.0545e-01],
        [-1.6054e+00,  1.1666e-01, -2.8253e-01,  ...,  1.0049e+00,
         -9.6251e-01,  1.8574e+00],
        [-1.0129e+00, -6.7703e-01,  3.7852e-01,  ...,  4.7505e-01,
         -1.4882e+00,  1.0448e+00]])

In [59]:
bad_indexes = target <= 3

In [60]:
bad_indexes.shape, bad_indexes.dtype, bad_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(20))

In [61]:
bad_data = data[bad_indexes]
bad_data.shape

torch.Size([20, 11])

In [62]:
bad_data = data[target <= 3]
mid_data = data[(target >3) & (target <7)]
good_data = data[target >= 7]

In [63]:
bad_mean = torch.mean(bad_data, dim = 0)
mid_mean = torch.mean(mid_data, dim = 0)
good_mean = torch.mean(good_data, dim = 0)

In [64]:
for i, args in enumerate(zip(col_list, bad_mean, mid_mean, good_mean)):
    print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i, *args))

 0 fixed acidity          7.60   6.89   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  34.55
 6 total sulfur dioxide 170.60 141.83 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.26  11.42


In [65]:
total_sulfur_threshold = 141.83
total_sulfur_data = data[:,6]
predicted_indexes = torch.lt(total_sulfur_data, total_sulfur_threshold)

In [66]:
predicted_indexes.shape, predicted_indexes.dtype, predicted_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(2727))

In [67]:
actual_indexes = target >5

In [68]:
actual_indexes.shape, actual_indexes.dtype, actual_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(3258))

In [69]:
n_matches = torch.sum(actual_indexes & predicted_indexes).item()
n_predicted = torch.sum(predicted_indexes).item()
n_actual = torch.sum(actual_indexes).item()

n_matches, n_matches/n_predicted, n_matches/n_actual

(2018, 0.74000733406674, 0.6193984039287906)

## Working with Time Series

In [70]:
bikes_numpy = np.loadtxt("git_rep/data/p1ch4/bike-sharing-dataset/hour-fixed.csv",
                        dtype= np.float32, delimiter=",", skiprows=1, 
                        converters={1: lambda x:float(x[8:10])})

In [71]:
bikes = torch.from_numpy(bikes_numpy)

In [72]:
bikes

tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 1.3000e+01,
         1.6000e+01],
        [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.0000e+00, 3.2000e+01,
         4.0000e+01],
        [3.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.0000e+00, 2.7000e+01,
         3.2000e+01],
        ...,
        [1.7377e+04, 3.1000e+01, 1.0000e+00,  ..., 7.0000e+00, 8.3000e+01,
         9.0000e+01],
        [1.7378e+04, 3.1000e+01, 1.0000e+00,  ..., 1.3000e+01, 4.8000e+01,
         6.1000e+01],
        [1.7379e+04, 3.1000e+01, 1.0000e+00,  ..., 1.2000e+01, 3.7000e+01,
         4.9000e+01]])

In [73]:
bikes.shape

torch.Size([17520, 17])

In [74]:
bikes.stride()

(17, 1)

In [75]:
daily_bikes = bikes.view(-1, 24, bikes.shape[1])

In [76]:
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 24, 17]), (408, 17, 1))

In [77]:
daily_bikes = daily_bikes.transpose(1,2)
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

In [78]:
first_day = bikes[:24].long()

In [79]:
first_day.shape

torch.Size([24, 17])

In [82]:
weather_onehot = torch.zeros(first_day.shape[0],4)

In [83]:
first_day[:,9]

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])

In [84]:
weather_onehot.scatter_(dim = 1, index = first_day[:,9].unsqueeze(1).long() - 1, value = 1.0)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [85]:
torch.cat((bikes[:24],weather_onehot),1)[:1]

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

In [86]:
daily_weather_onehot = torch.zeros(daily_bikes.shape[0],4,daily_bikes.shape[2])

In [87]:
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [89]:
daily_weather_onehot.scatter_(1, daily_bikes[:,9,:].long().unsqueeze(1)-1, 1.0)

tensor([[[1., 1., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0

In [90]:
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [91]:
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot),dim=1)

In [92]:
daily_bikes[:, 9 , :] = (daily_bikes[:, 9 , : ]-1.0)/3.0

In [93]:
temp = daily_bikes[:,10,:]
temp_min = torch.min(temp)
temp_max = torch.max(temp)

In [94]:
daily_bikes[:,10,:] = ((daily_bikes[:,10,:] - temp_min)/(temp_max - temp_min))

In [95]:
daily_bikes[:, 10, :] = ((daily_bikes[:,10,:] - torch.mean(temp))/ torch.std(temp))

## Representing Text

In [96]:
with open("git_rep/data/p1ch4/jane-austen/1342-0.txt", encoding='utf8') as f:
    text = f.read()

In [97]:
lines = text.split("\n")

In [98]:
line = lines[200]
line

'“Impossible, Mr. Bennet, impossible, when I am not acquainted with him'

In [99]:
letter_t = torch.zeros(len(line), 128)

In [100]:
letter_t.shape

torch.Size([70, 128])

In [101]:
for i, letter in enumerate(line.lower().strip()):
    letter_index = ord(letter) if ord(letter) < 128 else 0
    letter_t[i][letter_index] = 1

In [113]:
def clean_words(input_str):
    punctuation = '.,;:"!?“”_-'
    word_list = input_str.lower().replace("\n",' ').split()
    word_list = [word.strip(punctuation) for word in word_list]
    return word_list

In [114]:
words_in_line = clean_words(line)

In [115]:
line,words_in_line

('“Impossible, Mr. Bennet, impossible, when I am not acquainted with him',
 ['impossible',
  'mr',
  'bennet',
  'impossible',
  'when',
  'i',
  'am',
  'not',
  'acquainted',
  'with',
  'him'])

In [116]:
word_list = sorted(set(clean_words(text)))
word2idex_dict = {word:i for (i, word) in enumerate(word_list)}

In [117]:
len(word2idex_dict), word2idex_dict['impossible']

(7261, 3394)

In [118]:
word_t = torch.zeros(len(words_in_line),len(word2idex_dict))

In [119]:
for i, word in enumerate(words_in_line):
    word_index = word2idex_dict[word]
    word_t[i][word_index] = 1
    print('{:2} {:4} {}'.format(i, word_index,word))

 0 3394 impossible
 1 4305 mr
 2  813 bennet
 3 3394 impossible
 4 7078 when
 5 3315 i
 6  415 am
 7 4436 not
 8  239 acquainted
 9 7148 with
10 3215 him


In [120]:
print(word_t.shape)

torch.Size([11, 7261])
